# HACKATON

In [12]:
import pyModeS as pms
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from mpl_toolkits.basemap import Basemap
import os
import geopandas as gpd
from sphericalpolygon import Sphericalpolygon
from astropy import units as u
from tqdm import tqdm
from utils import qdrdist as qdr
from sklearn.metrics.pairwise import haversine_distances
import math

#for filtering aircraft entering barajas
def isinside(zone,lat,lon):
        return (zone[0][0] <= lon <= zone[1][0]) and (zone[0][1] <= lat <= zone[1][1])
    
#check for point of decent
def hdglimit(hdg): #if nan return false
    return 323>=hdg>=321 or 180>=hdg>=178

def altlimit(alt): #if nan return false
    return 5500>=alt>=3500

def merge_bds(values):
    unique_values = sorted(set(values))
    return ','.join(str(v) for v in unique_values)

def run_create(route, index):
    route_geo = './data/'
    #AIRPORT DATA
    runway = gpd.read_file(f'{route_geo}runways.geojson')
    threshold = gpd.read_file(f'{route_geo}thresholds.geojson')

    #SEPARATE COORDS
    threshold['lat'] = threshold.apply(lambda row: row.geometry.y,axis=1)
    threshold['lon'] = threshold.apply(lambda row: row.geometry.x,axis=1)
    threshold['hdg'] = threshold.apply(lambda row: row.orientation,axis=1)

    #VORTEX CATEGORY
    vortex = pd.read_csv(f'{route_geo}VORTEX.csv',sep=';')

    #AIRPORT ZONE
    barajas_zone = [[-3.773804,40.351118],[-3.314438,40.620516]]

    route = route
    #route = 'data/sample_scenario_00.parquet'

    df = pd.read_parquet(route)

    #list containing id of aircraft
    wake_flights_list = df[(df.bds=='08')&(df.wake_vortex.isin(vortex.id.tolist()))]['icao24'].value_counts().index.tolist()

    #Filtered df with all ADS-B data msg
    wake_df = df[df.icao24.isin(wake_flights_list)]

    #Only aircraft that have flown below 2000ft
    alt_flights_list = wake_df[wake_df.altitude<=4000]['icao24'].value_counts().index.tolist()
    w_a_df = wake_df[wake_df.icao24.isin(alt_flights_list)]

    #Only aircraft that have flown inside barajas airport zone
    w_a_df['inside'] = w_a_df.apply(lambda row: isinside(barajas_zone,row['lat_deg'],row['lon_deg']) 
                                    if pd.notna(row['lat_deg']) else False,axis=1)
    inside_flights_list = w_a_df[w_a_df.inside == True]['icao24'].value_counts().index.tolist()
    wai_df = w_a_df[w_a_df.icao24.isin(inside_flights_list)]

    #ONLY AIRCRAFT ON LANDING
    tmp = wai_df[(wai_df.bds=='05')&(wai_df.inside==True)]
    alt_min = tmp.loc[tmp.groupby('icao24')['ts'].idxmin(), ['icao24', 'altitude']].rename(columns={'altitude': 'alt_min'})
    alt_max = tmp.loc[tmp.groupby('icao24')['ts'].idxmax(), ['icao24', 'altitude']].rename(columns={'altitude': 'alt_max'})
    dif_df = alt_min.merge(alt_max, on='icao24')
    dif_df['dif'] = dif_df.alt_min-dif_df.alt_max
    landing_flights_list = dif_df[dif_df['dif']>0]['icao24'].value_counts().index.tolist()
    landing_df = wai_df[wai_df.icao24.isin(landing_flights_list)]
    landing_df = landing_df.sort_values(by='ts')
    landing_df['datetime'] = pd.to_datetime(landing_df["ts"], unit="ms").dt.strftime('%H:%M:%S')
    print(1)

    landing_df['alt_point'] = landing_df.apply(lambda row: altlimit(row.altitude),axis=1)
    landing_df['hdg_point'] = landing_df.apply(lambda row: hdglimit(row.track),axis=1)

    alt_hdg_df = landing_df[((landing_df['alt_point']==True)|(landing_df['hdg_point']==True))&((landing_df.bds=='09')|(landing_df.bds=='05'))].sort_values(by=['icao24','ts'])
    result = alt_hdg_df.groupby(['icao24', 'ts','datetime']).agg({
        'bds': merge_bds,
        'hdg_point': 'any',  # will prioritize True if any True exists
        'alt_point': 'any',  # same
    }).reset_index()
    merged_results = result[result['bds'] == '05,09'].drop_duplicates()

    acid = merged_results.groupby('icao24').min('ts').sort_values(by='icao24').index.tolist()
    times = merged_results.groupby('icao24').min('ts').sort_values(by='icao24').ts.tolist()

    indexes = []
    for i in range(len(acid)):
        row = landing_df[(landing_df.icao24==acid[i])&(landing_df.ts==times[i])&(landing_df.bds=='05')]
        indexes.append(row.index[0])

    landing_df['descent'] = landing_df.index.isin(indexes)

    
    
    def haversine_distance(p1, p2):
        p1 = [math.radians(_) for _ in p1]
        p2 = [math.radians(_) for _ in p2]
        dist = haversine_distances([p1, p2])
        return 6371000*dist[0][1]

    for _,row in tqdm(threshold.iterrows()):
        landing_df[f'dist_{row.orientation}'] = landing_df.apply(lambda x: haversine_distance((x.lat_deg,x.lon_deg), 
                                                            (row.lat,row.lon)) if x.bds == '05' else 9999,axis=1)

    landing_df['rwy_dist'] = landing_df.apply(lambda x: min(x.dist_18R,x.dist_32L,x.dist_32R,x.dist_18L),axis=1)

    model_acid = landing_df[landing_df.rwy_dist<600].icao24.value_counts().index.tolist()
    model_df = landing_df[landing_df.icao24.isin(model_acid)]

    min_dist = model_df.groupby('icao24').min('rwy_dist').reset_index()[['icao24','rwy_dist']]
    model_df['min_dist'] = model_df.apply(lambda x: min_dist[min_dist['icao24']==x.icao24].rwy_dist.iloc[0] == x.rwy_dist ,axis=1)

    model_df = model_df[(model_df.descent==True)|(model_df.min_dist==True)]
    model_df = model_df[model_df.groupby('icao24')['icao24'].transform('count') >= 2]
    model_df = model_df.sort_values(by=['icao24','ts'])

    #add heading via rwy
    mask = model_df[['dist_18R', 'dist_32L', 'dist_32R', 'dist_18L']].eq(model_df['rwy_dist'], axis=0)
    model_df['hdg'] = mask.dot(mask.columns).where(mask.any(axis=1), False)
    model_df['hdg'] = model_df['hdg'].str[5:7]+'0'
    model_df = model_df[['icao24','ts','datetime','lat_deg','lon_deg','hdg','altitude','tc']]
    print(3)
    codes = model_df.icao24.value_counts().index.tolist()
    data = []
    col = ['icao24','hdg','tc','ts1','datetime1','lat_deg1','lon_deg1','altitude1','ts2','datetime2','lat_deg2','lon_deg2','altitude2']
    for i in codes:
        tmp = model_df[model_df.icao24==i].sort_values(by='ts')
        ini = tmp.iloc[0]
        fin = tmp.iloc[1]
        data.append([ini.icao24, ini.hdg, ini.tc, ini.ts, ini.datetime, ini.lat_deg, ini.lon_deg, ini.altitude,
                                                  fin.ts, fin.datetime, fin.lat_deg, fin.lon_deg, fin.altitude])

    model_final = pd.DataFrame(data,columns=col)
    model_final['dif_t'] = model_final.ts2-model_final.ts1

    def feet_to_meters(f):
        return f*0.3048

    def distance(altitude,tc,point1,point2):
        is_alt_in_meters = True
        if 9 <= tc <= 18:
             is_alt_in_meters = False
        altitude_point = altitude if is_alt_in_meters else feet_to_meters(altitude) 
        distance_2d = haversine_distance(point1, point2)
        distance_3d = math.sqrt(altitude_point*altitude_point + distance_2d*distance_2d)
        return distance_3d
    print(4)
    model_final['distance_3d'] = model_final.apply(lambda x: distance(x.altitude1,x.tc,(x.lat_deg1,x.lon_deg1),
                                                                      (x.lat_deg2,x.lon_deg2)),axis=1)
    model_final['dif_t'] = model_final['dif_t']/1000 #ms to s
    model_final['gs'] = model_final['distance_3d']/model_final['dif_t'] #m/s

    wake_flights_list = df[(df.bds=='08')&(df.wake_vortex.isin(vortex.id.tolist()))]

    ids = model_final.icao24.value_counts().index.tolist()
    fin_wake = wake_flights_list[['icao24','wake_vortex']].drop_duplicates()
    fin_wake = fin_wake[fin_wake.icao24.isin(ids)]
    fin_wake['code'] = fin_wake.apply(lambda x: vortex[vortex.id==x.wake_vortex].code.iloc[0],axis=1)

    model_final['wake_code'] = model_final.apply(lambda x: fin_wake[fin_wake.icao24 == x.icao24].code.iloc[0] ,axis=1)
    model_final.to_csv(f"outputs/model_final_{index}.csv", index=False)

In [24]:
route_ini = 'data/engage-hackathon-2025/year=2024'
month = os.listdir(route_ini)
index = 0

limit = 304

for i in month:
    day = os.listdir(route_ini+'/'+i)
    tmp_m = route_ini+'/'+i
    for j in day:
        h = os.listdir(tmp_m+'/'+j)
        tmp_day = tmp_m+'/'+j
        for x in tqdm(h):
            index +=1
            if index>limit:
                run_create(tmp_day+'/'+x, index)    



  0%|          | 0/24 [00:00<?, ?it/s]

0
data/engage-hackathon-2025/year=2024/month=12/day=14/hour=11
si


C:\Users\1570074\AppData\Local\Temp\ipykernel_11432\3951986721.py:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  w_a_df['inside'] = w_a_df.apply(lambda row: isinside(barajas_zone,row['lat_deg'],row['lon_deg'])


1
2



0it [00:00, ?it/s]
1it [00:05,  5.47s/it]
2it [00:10,  5.36s/it]
3it [00:16,  5.46s/it]
4it [00:21,  5.39s/it]
C:\Users\1570074\AppData\Local\Temp\ipykernel_11432\3951986721.py:119: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  model_df['min_dist'] = model_df.apply(lambda x: min_dist[min_dist['icao24']==x.icao24].rwy_dist.iloc[0] == x.rwy_dist ,axis=1)


...

3
4
0
data/engage-hackathon-2025/year=2024/month=12/day=20/hour=10
si


C:\Users\1570074\AppData\Local\Temp\ipykernel_11432\3951986721.py:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  w_a_df['inside'] = w_a_df.apply(lambda row: isinside(barajas_zone,row['lat_deg'],row['lon_deg'])


1
2



0it [00:00, ?it/s]
1it [00:03,  3.54s/it]
2it [00:08,  4.04s/it]
  8%|▊         | 2/24 [00:22<04:04, 11.10s/it]


KeyboardInterrupt: 

In [23]:
index

303